### Проверка того, что наше api работает

Перед началом необходимо собрать docker-образ и запустить контейнер

1. Следовать инструкциям https://github.com/rabenkralle/Machine_learning/blob/master/lesson_9/README.md
2. Далее уже запускать код ниже

In [1]:
import pandas as pd
from sklearn.metrics import r2_score
from urllib import request, parse

X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [2]:
FEATURES = ['DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare', 'Floor', 'HouseFloor', 'HouseYear']

In [3]:
X_test[FEATURES].head(3)

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear
0,22,1.0,42.794118,27.816177,3.0,11.0,17.0,1977.0
1,11,1.0,42.535158,21.921874,8.0,7.0,9.0,1987.0
2,122,3.0,75.222209,46.157077,10.0,10.0,14.0,1996.0


In [4]:
import urllib.request
import json      


def get_prediction(x):
    district, rooms, square, lifesquare, kitchensquare, floor, housefloor, houseyear = x
    body = {'DistrictId':[district],
            'Rooms': [rooms],
            'Square': [square],
            'LifeSquare': [lifesquare],
            'KitchenSquare': [kitchensquare],
            'Floor': [floor],
            'HouseFloor': [housefloor],
            'HouseYear': [houseyear]}

    myurl = "http://0.0.0.0:8180/predict"
    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    
    response = urllib.request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())['predictions']

In [5]:
%%time
predictions = X_test[FEATURES].iloc[:500].apply(lambda x: get_prediction(x), 1)

CPU times: user 346 ms, sys: 79.5 ms, total: 425 ms
Wall time: 19.5 s


In [6]:
predictions

0      141312.114611
1      170229.119934
2      336623.491489
3      216034.886950
4       83586.607735
           ...      
495    181555.885473
496    386405.481270
497    121231.846633
498    235258.885952
499    337588.600437
Length: 500, dtype: float64

In [7]:
r2_score(y_true=y_test.iloc[:500], y_pred=predictions)

0.7115615023534146